# Gestor de investimento
---

In [1]:
import pandas as pd
import numpy as np
import os


# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_abr = pd.read_excel('../datasets/PosicaoHistorico_200091-3.xls')
df_jan = pd.read_excel('../datasets/PosicaoHistorico_200091-4.xls')

WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)


In [2]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtde Projetado', 'Qtde Dia', 
                                    'Qtde Garantia BOV', 'Qtde Garantia BMF', 
                                    'Qtde Estruturados', 'Liq Termo', 
                                    'Qtde Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock

In [3]:
files = os.listdir('../datasets/')

stocks = pd.DataFrame()

for file_name in files:
    
    if file_name == '.DS_Store':
        continue
    
    df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    month = int(date_position.dt.month.values)
    year = int(date_position.dt.year.values)

    #if file_name == 'PosicaoHistorico_200091-12.xls':
       # display(df[~df['Unnamed: 2'].isnull()].head(10))

    #print(str(year) + '/' + str(month) + '   ' + file_name)
    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = str(year) + '/' + str(month)
    stocks = stocks.append(df_stocks, ignore_index=True)

display(stocks.sort_values('period', ascending=False))

WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)
WARNING *** file size (65692) not 512 + multiple of sector size (512)
WARNING *** file size (70812) not 512 + multiple of sector size (512)
WARNING *** file size (65696) not 512 + multiple of sector size (512)
WARNING *** file size (66716) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (65180) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)


,Papel,Qtd Disponivel,Qtde Projetado,Qtde Dia,Qtde Garantia BOV,Qtde Garantia BMF,Qtde Estruturados,Liq Termo,Qtde Total,Cotacao,Financeiro,period
0,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020/4
2,MGLU3,800,0,0,0,0,0,0,800,49.70,"39,760.00",2020/4
3,PETR4,400,0,0,0,0,0,0,400,18.05,"7,220.00",2020/4
4,WEGE3,300,0,0,0,0,0,0,300,39.94,"11,982.00",2020/4
1,JHSF3,1600,0,0,0,0,0,0,1600,4.16,"6,656.00",2020/4
8,JHSF3,1600,0,0,0,0,0,0,1600,3.50,"5,600.00",2020/3
11,WEGE3,300,0,0,0,0,0,0,300,33.57,"10,071.00",2020/3
10,PETR4,400,0,0,0,0,0,0,400,13.99,"5,596.00",2020/3
9,MGLU3,800,0,0,0,0,0,0,800,38.99,"31,192.00",2020/3
7,BPAC11,200,0,0,0,0,0,0,200,33.24,"6,648.00",2020/3


In [45]:
stocks.head(10)

,Papel,Qtd Disponivel,Qtde Projetado,Qtde Dia,Qtde Garantia BOV,Qtde Garantia BMF,Qtde Estruturados,Liq Termo,Qtde Total,Cotacao,Financeiro,period
0,MGLU3,800,0,0,0,0,0,0,800,47.66,"38,128.00",2019/12
1,MGLU3,800,0,0,0,0,0,0,800,44.64,"35,712.00",2019/10
2,MGLU3,800,0,0,0,0,0,0,800,45.03,"36,024.00",2019/11
3,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019/9


In [21]:
df_abr[~df_abr['Unnamed: 2'].isnull()].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
16,nan,NaN,Ações,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,74078,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
20,nan,NaN,Papel,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
24,nan,NaN,BPAC11,nan,nan,nan,nan,nan,nan,nan,200,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,200,nan,NaN,nan,NaN,nan,NaN,nan,42.30,NaN,nan,nan,nan,nan,NaN,"8,460.00",NaN
26,nan,NaN,JHSF3,nan,nan,nan,nan,nan,nan,nan,1600,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,1600,nan,NaN,nan,NaN,nan,NaN,nan,4.16,NaN,nan,nan,nan,nan,NaN,"6,656.00",NaN
28,nan,NaN,MGLU3,nan,nan,nan,nan,nan,nan,nan,800,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,800,nan,NaN,nan,NaN,nan,NaN,nan,49.70,NaN,nan,nan,nan,nan,NaN,"39,760.00",NaN
30,nan,NaN,PETR4,nan,nan,nan,nan,nan,nan,nan,400,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,400,nan,NaN,nan,NaN,nan,NaN,nan,18.05,NaN,nan,nan,nan,nan,NaN,"7,220.00",NaN
32,nan,NaN,WEGE3,nan,nan,nan,nan,nan,nan,nan,300,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,300,nan,NaN,nan,NaN,nan,NaN,nan,39.94,NaN,nan,nan,nan,nan,NaN,"11,982.00",NaN
35,nan,NaN,Opções,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
37,nan,NaN,Termos,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,n

In [22]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
0,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
1,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,Histórico de Posição Consolidada,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
2,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Mês: Maio/2019,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
3,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Data de referência: 31/05/2019,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
4,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
5,nan,Gabriel Machado Lopes Goncalves,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,CPF/CNPJ,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,980.742.371-68,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
6,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
7,nan,"Rua Barata Ribeiro, 160 - Ap 908\nRio De Janei...",NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,Conta XP,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,200091,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
8,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,